## Integration

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
from BZI.pseudopots import Al_lat_const, Al_rlat_vecs, Al_PP
from BZI.integration import rectangular_method, rectangle_method, simple_fl
from BZI.symmetry import make_ptvecs, make_rptvecs
from BZI.sampling import make_grid
from BZI.plots import ScatterPlotMultiple
from BZI.convergence import Convergence
from math import ceil

Find the "solution" to the Fermi level and total energy

In [6]:
grid_type = "fcc"
grid_centering = "face"
grid_const = Al_lat_const*20
grid_consts = [grid_const]*3
grid_angles = [np.pi/2]*3
rgrid_vecs = make_ptvecs(grid_centering, grid_consts, grid_angles)
grid_vecs = make_rptvecs(rgrid_vecs)
offset = .5*np.sum(grid_vecs,1)
grid = make_grid(Al_rlat_vecs, grid_vecs, offset)
neigvals = 6
nvalence = 3

In [7]:
fermi_level = simple_fl(Al_PP,grid,neigvals,nvalence)

In [8]:
fermi_level

11.579736297413053

In [9]:
rectangle_method(Al_PP, Al_rlat_vecs, grid_vecs, offset, fermi_level, neigvals)

(8000, 22.784184574840808)